In [1]:
import xarray as xr
import now
import xscale.signal.fitting as xfit
from dask.distributed import LocalCluster, Client

## Starting a Local Cluster with dask.distributed
Here we will define a local cluster on the NCI Virtual Desktop and choose to use multithreading on the 8 cores available. Multithreading is achieved by setting `processes` to `False`. Since dask workers may write exceeding bits of data that not fit in the memory to the drive, we set the `local_dir` to the /g/data1/e14. More information about how to set a Local Cluster with dask is given [here](http://docs.dask.org/en/latest/setup/single-distributed.html). 

In [2]:
local_dir = "/g/data1/e14/gs9353/dask-workers/"
cluster = LocalCluster(processes=False, local_dir=local_dir)

Creating a scheduler linked to the previous cluster can be done creating an instance of the `Client` object. Calling `Client()` also create an associated dask taskboard, which is accessible via an IP address.

In [3]:
client = Client(cluster)
client

Client Scheduler: inproc://10.0.64.3/930/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


## Choosing a configuration file

In [5]:
config_file = '/g/data1/e14/gs9353/CONFIG/ecl_study.ini'

## Compute the mean state

In [7]:
test = now.io.open_nemo_griddata_from_zarr(config_file, grid='T')

In [9]:
mean_test = test.mean('time_counter').compute()
print(mean_test)

/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


<xarray.Dataset>
Dimensions:     (simulation: 3, x: 431, y: 289)
Coordinates:
    nav_lat     (y, x) float32 -43.27347 -43.37204 ... 14.78277 14.848713
    nav_lon     (y, x) float32 82.76912 83.00413 ... -175.2138 -175.04564
  * simulation  (simulation) <U19 'Present' 'No Current Feedback' 'Future'
Dimensions without coordinates: x, y
Data variables:
    20d         (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    blt         (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    depti       (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    mld_dt02    (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    mldkz5      (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    mldr10_1    (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    mldr10_3    (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    pycndep     (simulation, y, x) float32 nan nan nan nan ... nan nan nan nan
    rsntds    

### NEMO

In [5]:
for grid in ['U', 'V']:
    griddata = now.io.open_nemo_griddata_from_zarr(config_file, grid=grid)
    griddata_mean = griddata.mean('time_counter')
    output_name = 'CURRENT_FEEDBACK_NEMO_1990-2008_1d_grid_%s_3D_mean.nc' %grid
    now.io.to_postprocess(griddata_mean, config_file, output_name, type='netcdf')

/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


## Compute the linear trend

In [20]:
for grid in ['U', 'V', 'T']:
    griddata = now.io.open_nemo_griddata_from_zarr(config_file, grid=grid)
    if grid is 'T':
        griddata = griddata.drop(('time_maximum_1d', 'time_minimum_1d'))
    griddata_trend, _ = xfit.linreg(griddata, dim='time_counter')
    output_name = 'ECL_NEMO_1990-2008_1d_grid_%s_2D_trend_slope.zarr' %grid
    now.io.to_postprocess(griddata_trend, config_file, output_name, type='zarr')

/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/linalg.py:1095: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.array([0, 1], dtype=b.dtype))
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/linalg.py:1095: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.array([0, 1], dtype=b.dtype))
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/linalg.py:1095: FutureWarning: `rcond` parameter will change to the default of mac

## Compute the climatological cycle

### NEMO

In [21]:
for grid in ['U', 'V', 'T']:
    griddata = now.io.open_nemo_griddata_from_zarr(config_file, grid=grid)
    griddata_seasonal_cycle = griddata.drop(('nav_lon', 'nav_lat'))\
                                      .groupby('time_counter.dayofyear')\
                                      .mean('time_counter')
    griddata_seasonal_cycle = griddata_seasonal_cycle.assign_coords(nav_lon=griddata['nav_lon'].chunk(), 
                                                                    nav_lat=griddata['nav_lat'].chunk())
    output_name = 'ECL_NEMO_1990-2008_1d_grid_%s_2D_seasonal_cycle.zarr' %grid
    now.io.to_postprocess(griddata_seasonal_cycle, config_file, output_name, type='zarr')

/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: divide by zero encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: divide by zero encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/gs9353/miniconda3/envs/nemo_env/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://10.0.64.22/10667/18 remote=inproc://10.0.64.22/10667/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://10.0.64.22/1

## Make a smooth version of the seasonal cycle
Because the seasonal cycle computed over 1 day outputs is noisy, a smoothed version is made using a simple boxcar window.

In [22]:
for grid in ['U', 'V', 'T']:
    griddata_seasonal_cycle = xr.open_zarr('/g/data1/e14/gs9353/POST-PROCESS/ECLs/ECL_NEMO_1990-2008_1d_grid_%s_2D_seasonal_cycle.zarr/'% grid)
    win = griddata_seasonal_cycle.window
    win.set(n=21, dim='dayofyear', chunks={'dayofyear': 35})
    output_name = 'ECL_NEMO_1990-2008_1d_grid_%s_2D_seasonal_cycle_20d_smooth.zarr' %grid
    now.io.to_postprocess(win.convolve(), config_file, output_name, type='zarr', o)